# The guessing Game

Concept of this game is quite easy. Two models interact with each other to guess noun.

### Prompt preparation

In [76]:
HOST_TEMPLATE = """
You are the host of a game where a player asks questions about
a thing to guess what it is.

Write the name of a thing. It must be a common object.
It must be a single word. Do not write anything else. 
Only write the name of the thing with no punctuation.

Here is a list of things you cannot use:
{history}
"""

QUESTION_TEMPLATE = """
You are a player in a game where you need to ask Yes/No questions about 
a thing and guess what it is.

The thing is a common object. It is a single word.

Here are the questions you have already asked:

{observations}

You only have {questions_left} questions left to ask. You want to guess
in as few questions as possible.
If there's only 1 question left, 
you must make a guess or you'll lose the game. Be aggressive and try to
guess the thing as soon as possible.

Do not ask questions that you have already asked before.

Only binary question are allowed. The question must be answered
with a Yes/No.

Be as concise as possible when asking a question. Do not announce that you
will ask the question. Do not say "Let's get started", or introduce your 
question. Just write the question.

Examples of good questions:

- Is it a fruit?
- Is it bigger than a car?
- Is it alive?

Examples of bad questions:

- Can I ask a question?
- Can you tell me more about the thing?
- What is the thing?
- How does the thing look like?
"""

ANSWER_TEMPLATE = """
You are the host of a game where a player asks questions about
a {concept} trying to guess what it is.

The player has asked you the following question: {question}.

If the player guessed that the thing is "{concept}" doesn't mather if written lover case or upper case, answer with the word "GUESSED".
If the question refers specific to "{concept}", answer with the word "GUESSED".

If the player didn't guessed, answer the question with a 
simple Yes or No. Do not say anything else. Do not use any punctuation.
"""

### Helper to initialize model

In [77]:
from httpx import ConnectError
from langchain_core.language_models.llms import BaseLLM
from langchain_ollama import OllamaLLM
from ollama import ResponseError


def create_model(model_name: str = "llama3.2") -> BaseLLM:
    model = OllamaLLM(model=model_name)
    try:
        print(model.invoke("Say hello"))
    except ResponseError as e:
        print("Ollama model not found. Pull it or change.")
        print(e)
        exit(1)
    except ConnectError as e:
        print("Ollama application is not running. Starting it or download.")
        print(e)
        exit(1)

    return model

### Player Class

Player represents model with:
* ability to initialize game with concept
* ask question with some kind of history
* answer question based on output

In [78]:
from langchain_core.language_models.llms import BaseLLM
from langchain_core.prompts.base import BasePromptTemplate
from langchain_core.output_parsers.base import BaseOutputParser


class Player:
    def __init__(self, model: BaseLLM):
        self.observations = []
        self.model = model
        self.concept = None
        self.history = []

    def initialize_host(self, prompt_template: BasePromptTemplate, output_parser: BaseOutputParser):
        prompt = prompt_template.from_template(HOST_TEMPLATE)
        chain = prompt | self.model | output_parser

        self.concept = chain.invoke({"history": self.history})
        self.history.append(self.concept)

        print(f"Concept: {self.concept}")

    def initialize_player(self):
        self.observations = []

    def ask_question(
        self, prompt_template: BasePromptTemplate, output_parser, questions_left
    ):
        prompt = prompt_template.from_template(QUESTION_TEMPLATE)
        chain = prompt | self.model | output_parser

        question = chain.invoke(
            {
                "observations": "\n".join(self.observations),
                "questions_left": questions_left,
            }
        )

        self.observations.append(question)

        return question

    def answer_question(
        self, prompt_template: BasePromptTemplate, output_parser, question
    ):
        prompt = prompt_template.from_template(ANSWER_TEMPLATE)
        chain = prompt | self.model | output_parser

        return chain.invoke({"concept": self.concept, "question": question})

    def add_observation(self, question: str, answer: str):
        self.observations.append(f"Question: {question}. Answer: {answer}")


#### Testing player object

In [ ]:
model = create_model()

In [ ]:
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

player = Player(model=model)
player.initialize_host(prompt_template=PromptTemplate, output_parser=StrOutputParser())

In [ ]:
question = player.ask_question(prompt_template=PromptTemplate, output_parser=StrOutputParser(), questions_left=20)
question

In [ ]:
player.answer_question(prompt_template=PromptTemplate, output_parser=StrOutputParser(), question=question)

### Game Class

In [83]:
from langchain_core.language_models.llms import BaseLLM
from langchain_core.prompts.base import BasePromptTemplate
from langchain_core.output_parsers.base import BaseOutputParser


class Game:
    def __init__(
        self,
        model1: BaseLLM,
        model2: BaseLLM,
        prompt_template: BasePromptTemplate,
        output_parser: BaseOutputParser,
        rounds: int = 3,
        questions: int = 20,
    ):
        self.model1 = model1
        self.model2 = model2
        self.prompt_template = prompt_template
        self.output_parser = output_parser
        self.rounds = rounds
        self.questions = questions

    def start(self):
        players = {
            "0": {
                "player": Player(self.model1),
                "score": 0,
            },
            "1": {
                "player": Player(self.model2),
                "score": 0,
            },
        }

        host_index = 0
        for round in range(self.rounds):
            print(f"Round {round + 1}. Player {host_index + 1} is the host.")

            player_index = 1 - host_index
            if self._play(
                players[str(host_index)]["player"], players[str(player_index)]["player"]
            ):
                print(f"Player {player_index + 1} guessed correctly.")
                players[str(player_index)]["score"] += 1
            else:
                print(f"Player {player_index + 1} didn't guess correctly.")
                players[str(host_index)]["score"] += 1

            host_index = 1 - host_index

        print("Final score:")
        print(f"Player 1: {players['0']['score']}")
        print(f"Player 2: {players['1']['score']}")

    def _play(self, host: Player, player: Player):
        host.initialize_host(self.prompt_template, self.output_parser)
        player.initialize_player()
        for question_index in range(self.questions):
            question = player.ask_question(
                self.prompt_template,
                self.output_parser,
                self.questions - question_index,
            )
            answer = host.answer_question(
                self.prompt_template, self.output_parser, question
            )

            print(f"Question {question_index + 1}: {question}. Answer: {answer}")

            player.add_observation(question, answer)

            if "guessed" in answer.lower():
                return True

        return False

## Game starts here

> Remember to run all previous steps to correctly run game.

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

game = Game(
    model1=create_model(model_name="llama3.2"),
    model2=create_model(model_name="llama3.2"),
    rounds=7,
    prompt_template=PromptTemplate,
    output_parser=StrOutputParser(),
)

game.start()